In [1]:
import os
import numpy as np
import pandas as pd
os.chdir("../")
from src.data import ImportData, getDataset


In [3]:
df_transaction = getDataset()

In [4]:
df_transaction.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,1/12/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12.0,1/12/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12.0,1/12/2009 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0,1/12/2009 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0,1/12/2009 7:45,1.25,13085.0,United Kingdom


In [5]:
df_transaction.isna().sum()

Invoice             1
StockCode           1
Description      4383
Quantity            1
InvoiceDate         1
Price               1
Customer ID    240030
Country             1
dtype: int64